## Load latest headlines

In [1]:
# ========== Bootstrap: ensure required Python packages are present =========
import importlib, subprocess, sys
from typing import Optional

def _ensure(pkg_name: str, import_name: Optional[str] = None):
    try:
        importlib.import_module(import_name or pkg_name)
    except ModuleNotFoundError:
        subprocess.check_call([sys.executable, '-m', 'pip', 'install', pkg_name])
    finally:
        globals()[import_name or pkg_name] = importlib.import_module(import_name or pkg_name)

_ensure('pandas')
print('All dependencies ready.\n')


All dependencies ready.



In [2]:
from pathlib import Path
archive = Path('archive')
archive.mkdir(exist_ok=True)
import pandas as pd
latest = pd.read_csv('../headlines/latest.csv')
latest.head()

,pubdate,source,title,link
0,2025-09-23-23-50-48 +0000,nypost,Curtis Sliwa demands equal time from ABC with ...,https://nypost.com/2025/09/23/us-news/curtis-s...
1,2025-09-23-23-47-25 +0000,nypost,NY judge found ‘credible evidence’ that Sen. R...,https://nypost.com/2025/09/23/us-news/ny-judge...
2,2025-09-23-23-46-11 +0000,nyt,Live Updates: After Meeting With Zelensky at U...,https://www.nytimes.com/live/2025/09/23/world/...
3,2025-09-23-23-39-02 +0000,nyt,"Claudia Cardinale, Actress Who Was ‘Italy’s Gi...",https://www.nytimes.com/2025/09/23/movies/clau...
4,2025-09-23-23-34-55 +0000,nyt,"In a Sudden Shift, Trump Says Ukraine Can Win ...",https://www.nytimes.com/2025/09/23/us/politics...


## Calculate word frequencies

In [3]:
import re
from collections import Counter
from datetime import datetime

with open('exclude.txt') as f:
    stop_words = set(w.strip() for w in f if w.strip())
words = re.findall(r'[A-Za-z]+', ' '.join(latest['title']).lower())
filtered = [w for w in words if w not in stop_words and len(w) > 1]
counts = Counter(filtered)
score_df = (
    pd.DataFrame(counts.items(), columns=['word','score'])
    .sort_values('score', ascending=False)
)
score_df[['score','word']].to_csv('scores.csv', index=False)
timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')
score_df[['score','word']].to_csv(archive / f'scores-{timestamp}.csv', index=False)
score_df.head()


/tmp/ipykernel_2379/2220774760.py:15: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')


,word,score
33,trump,79
276,kimmel,18
275,jimmy,14
319,president,12
243,un,12


## Rank headlines by score

In [4]:
word_scores = dict(score_df[['word','score']].values)
latest['score'] = latest['title'].apply(
    lambda t: sum(
        word_scores.get(w.lower(), 0)
        for w in re.findall(r'[A-Za-z]+', t)
        if len(w) > 1
    )
)
ranked = latest.sort_values('score', ascending=False)
ranked[['score','pubdate','source','title','link']].to_csv('rank.csv', index=False)
ranked[['score','pubdate','source','title','link']].to_csv(archive / f'rank-{timestamp}.csv', index=False)
ranked.head()


,pubdate,source,title,link,score
309,2025-09-23-03-02-00 +0000,wsj,In labeling Tylenol use in pregnant women as a...,https://www.wsj.com/politics/in-targeting-a-co...,145
106,2025-09-23-19-35-00 +0000,wsj,President Trump attacked global immigration an...,https://www.wsj.com/politics/trump-says-migrat...,142
130,2025-09-23-18-26-27 +0000,nypost,Trump supports NATO countries shooting down Ru...,https://nypost.com/2025/09/23/us-news/trump-su...,124
240,2025-09-23-12-21-27 +0000,nypost,Trump to tout US as defender of Western civili...,https://nypost.com/2025/09/23/us-news/trump-to...,124
323,2025-09-23-01-00-00 +0000,wsj,The worker visas that President Trump says now...,https://www.wsj.com/economy/jobs/h1b-visas-wor...,119


## Select top headlines

In [5]:
top_rows = []
working = word_scores.copy()
remaining = latest.copy()
for _ in range(10):
    ranked_loop = remaining.assign(score=remaining['title'].apply(
        lambda t: sum(working.get(w.lower(), 0)
                      for w in re.findall(r'[A-Za-z]+', t)
                      if len(w) > 1)
    )).sort_values('score', ascending=False)
    if ranked_loop.empty:
        break
    top_story = ranked_loop.iloc[0]
    top_rows.append(top_story[['score','pubdate','source','title','link']])
    words = set(re.findall(r'[A-Za-z]+', top_story['title'].lower()))
    for w in words:
        working.pop(w, None)
    remaining = remaining.drop(top_story.name)
top_df = pd.DataFrame(top_rows)
top_df.to_csv('top.csv', index=False)
top_df.to_csv(archive / f'top-{timestamp}.csv', index=False)
top_df


,score,pubdate,source,title,link
309,145,2025-09-23-03-02-00 +0000,wsj,In labeling Tylenol use in pregnant women as a...,https://www.wsj.com/politics/in-targeting-a-co...
192,72,2025-09-23-15-26-23 +0000,latimes,Nexstar and Sinclair TV stations will not run ...,https://www.latimes.com/entertainment-arts/bus...
106,51,2025-09-23-19-35-00 +0000,wsj,President Trump attacked global immigration an...,https://www.wsj.com/politics/trump-says-migrat...
141,44,2025-09-23-17-53-00 +0000,wsj,Fed Chair Jerome Powell said he judged the Fed...,https://www.wsj.com/economy/central-banking/fe...
101,43,2025-09-23-19-52-00 +0000,wsj,"Canada Needs to Chart New Economic Course, Ban...",https://www.wsj.com/articles/canada-needs-to-c...
170,43,2025-09-23-16-52-00 +0000,wsj,The Secret Service said it dismantled a networ...,https://www.wsj.com/politics/national-security...
307,40,2025-09-23-03-48-28 +0000,bbc,NZ woman found guilty of killing her two child...,https://www.bbc.com/news/articles/c2lxpvj19xdo...
155,38,2025-09-23-17-29-24 +0000,nypost,Megyn Kelly slams ‘coward’ Van Jones for brand...,https://nypost.com/2025/09/23/media/megyn-kell...
288,35,2025-09-23-08-03-21 +0000,nypost,Ukraine seeks support as Zelensky visits US to...,https://nypost.com/2025/09/23/world-news/ukrai...
312,33,2025-09-23-02-51-00 +0000,wsj,The team of federal economists and researchers...,https://www.wsj.com/politics/policy/usda-puts-...


## Create JSON version of top headlines

In [6]:
import pandas as pd
pd.read_csv('top.csv').to_json('top.json', orient='records', indent=2)
